In [ ]:
!pip install -qU gradio python-dotenv langchain-upstage

In [ ]:
%load_ext dotenv
%dotenv
# set UPSTAGE_API_KEY

In [ ]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
import gradio as gr

from langchain_upstage import ChatUpstage, UpstageEmbeddings, UpstageLayoutAnalysisLoader
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.schema import AIMessage, HumanMessage
from langchain_text_splitters import (
    Language,
    RecursiveCharacterTextSplitter,
)
from langchain_chroma import Chroma

llm = ChatUpstage(streaming=True)

In [ ]:
# More general chat
chat_with_history_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant.\nContext: {context}"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{message}"),
    ]
)

In [ ]:
chain = chat_with_history_prompt | llm | StrOutputParser()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter.from_language(
    chunk_size=1000, chunk_overlap=100, language=Language.HTML
)

In [ ]:
def chat(message, history, retriever):
    result_docs = ""
    if retriever:
        result_docs = retriever.invoke(message)

    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))

    generator = chain.stream({"context": result_docs,"message": message, "history": history_langchain_format})

    assistant = ""
    for gen in generator:
        assistant += gen
        yield assistant

In [ ]:
def file_upload(file):
    layzer = UpstageLayoutAnalysisLoader(file, output_type="html")
    docs = layzer.load()
    splits = text_splitter.split_documents(docs)
    print(len(splits))
    vectorstore = Chroma.from_documents(
        documents=splits,
        embedding=UpstageEmbeddings(model="solar-embedding-1-large"),
    )
    retriever = vectorstore.as_retriever()

    return file, retriever

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("# Solar Chatbot")
    gr.Markdown("Upstage Solar Chatbot",)
    with gr.Row():
        with gr.Column():
            file = gr.File()
            retreiver = gr.State()
        with gr.Column():
            chatbot = gr.ChatInterface(
                chat,
                examples=[
                    ["How to eat healthy?"],
                    ["Best Places in Korea"],
                    ["How to make a chatbot?"],
                ],
                additional_inputs=retreiver
            )
    chatbot.chatbot.height = 300
    file.upload(file_upload, file, [file, retreiver])

In [ ]:
if __name__ == "__main__":
    demo.launch()